In [1]:
from chain_logging import langsmith
from dotenv import load_dotenv

load_dotenv()

langsmith("langchain_RAG")


LangSmith 추적을 시작합니다.
[프로젝트명]
langchain_RAG


### 대화 내용을 기억하는 기능을 추가

In [3]:
import os
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from vectorstore_manager import (
    load_vectorstore,
    create_vectorstore,
    create_conversaion_memory,
)
from config import PDF_PATH, VECTORSTORE_PATH, MODEL_PATH, LOCAL_MODEL_PATH, MODEL_NAME
from utils import get_embeddings, format_docs_with_print, create_prompt
from load_model import load_model, create_model

# Load or create vectorstore
if os.path.exists(VECTORSTORE_PATH):
    print("Loading existing vectorstore...")
    embeddings = get_embeddings()
    loaded_vectorstore = load_vectorstore(VECTORSTORE_PATH, embeddings)
else:
    print("Vectorstore not found. Creating a new one...")
    embeddings = get_embeddings()
    loaded_vectorstore = create_vectorstore(PDF_PATH, VECTORSTORE_PATH, embeddings)

# 대화를 기억할 메모리 객체 생성
memory = create_conversaion_memory("chat_history", "answer")

# 단계 5: 리트리버 생성
k = 2
retriever = loaded_vectorstore.as_retriever(search_kwargs={"k": k})

# 단계 6: 프롬프트 생성
prompt = create_prompt()

# 단계7: 언어모델 생성
# Ollama 모델을 불러옵니다.
llm = ChatOllama(model=MODEL_PATH, temperature=0)

# if os.path.exists(LOCAL_MODEL_PATH):
#     print("Loading existing Local Model...")
#     llm = load_model(LOCAL_MODEL_PATH)
# else:
#     print("Local Model not found. Creating a new one...")
#     create_model(MODEL_NAME,LOCAL_MODEL_PATH)
#     llm = load_model(LOCAL_MODEL_PATH)


# 단계 8: 체인 생성
# 단계 8: 체인 생성
rag_chain = (
    {
        "context": retriever | format_docs_with_print,
        "question": RunnablePassthrough(),
        "chat_history": lambda x: memory.load_memory_variables({})["chat_history"],
    }
    | prompt
    | llm
    | StrOutputParser()
)

Loading existing vectorstore...


c:\Users\SSAFY\ssafy\자율프로젝트\S11P31S102\AI\chain\vectorstore_manager.py:42: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [6]:
from chain_output_stream import stream_response

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
# question = "연차는 어떻게 사용할 수 있나요?"
# question = "유류비 청구는 어떻게 할 수 있나요? 제가 3박 4일로 부산에 갔다오는데 비용은 대충 어떻게 계산하죠?"
question = "아까 물어본 질문을 다시 알려주세요"

response = rag_chain.stream(question)

# 결과 출력
print(f"[HUMAN]\n{question}\n")
print("===" * 20)
answer = stream_response(response, return_output=True)

# 메모리에 대화 내용을 저장
memory.save_context({"question": question}, {"answer": answer})



[HUMAN]
아까 물어본 질문을 다시 알려주세요


=== Retrieved Documents ===

Document 1:
..\kisa_pdf\3_04_감사규칙(230605).pdf |  page: 20
[별지제8호]<개정2013.5.8>
질문서
제목:
질문사항 답변사항
홍길동귀하 홍길동귀하
다음사항에대하여 20년00월00일까지
답변하여주시기바랍니다 .다음과같이답변합니다 .
20년00월00일
소속:
20년00월00일 직급 (직책 ):
감사실장 :  성명:홍길동
--------------------------------------------------

Document 2:
..\kisa_pdf\4_46_118상담센터 상담사 보호에 관한 업무 운영지침(221223).pdf |  page: 8
3.민원요지불명
구분내용 처리프로세스 세부응대프로세스
1단계민원응대및
경고후종료①1차경고
②2차경고
③3차경고및IVR응대<1차경고>
“선생님 ,문의하실내용을말씀해주세요 .그렇지않으면내용이 
파악되지않아더이상상담진행이어렵습니다 .”
<2차경고>
“선생님 ,문의하실내용을말씀해주세요 .그렇지않으면내용이 
파악되지않아더이상상담진행이어렵습니다 .”
<3차경고>
“문의내용이파악되지않아상담이불가합니다 .통화종료하
겠습니다 .”
→멘트구사후IVR전환
<IVR경고멘트자동송출 >
문의내용이파악되지않아상담이불가합니다 .상담사의권익
보호를위하여 ,선생님의상담이 7일간제한됨을알려드립니다 .
2단계악성민원등록①상담사상담후팀장보고
②팀장내용확인후상담APP
강성민원등록(민원요지불명 )강성민원등록시,동일번호 7일간이용정지
3-1단계이용정지 IVR연결불가멘트송출<IVR연결불가멘트자동송출 >
상담사의권익보호를위하여 ,선생님의상담이제한됨을알려
드립니다 .
3-2단계정지해제이용정지 8일차해제이용정지 8일차해제
※재발시반복이용정지
4.반복‧
--------------------------------------------------


Anser: 

홍길동 귀하, 아까 물어보신 질문은 유류비 청구와 연차